# Ungraded Lab: Feature Engineering with Accelerometer Data

Este cuaderno demuestra cómo preparar datos de series temporales tomados de un acelerómetro. Para este ejemplo utilizaremos el [WISDM Human Activity Recognition Dataset](http://www.cis.fordham.edu/wisdm/dataset.php). Este conjunto de datos puede utilizarse para predecir la actividad que realiza un usuario a partir de un conjunto de valores de aceleración registrados desde el acelerómetro de un smartphone.

El conjunto de datos consta de datos del acelerómetro en los ejes x, y y z registrados para 36 usuarios diferentes. Un total de 6 actividades: "Caminar", "Correr", "Subir escaleras", "Bajar escaleras", "Sentarse" y "Estar de pie". Los sensores tienen una frecuencia de muestreo de 20 Hz, lo que significa que se registran 20 observaciones por segundo.

## Install Packages

As with the previous lab, you will install the `tensorflow_transform` Python package and its dependencies.


In [ ]:
!pip install tensorflow_transform==1.4.0
!pip install apache-beam==2.39.0

**Nota: En Google Colab, es necesario reiniciar el tiempo de ejecución en este punto para finalizar la actualización de los paquetes que acaba de instalar. Puede hacerlo haciendo clic en el botón "Reiniciar el tiempo de ejecución" al final de la celda de salida anterior (después de la instalación), o seleccionando "Tiempo de ejecución > Reiniciar el tiempo de ejecución" en la barra de menús. **Por favor, no pases a la siguiente sección sin reiniciar.** También puedes ignorar los errores de incompatibilidad de versiones de algunos de los paquetes incluidos porque no los usaremos en este cuaderno.*

## Imports

La ejecución de las importaciones que se indican a continuación no debería mostrar ningún error. De lo contrario, reinicie su tiempo de ejecución o vuelva a ejecutar la celda de instalación del paquete anterior.

In [ ]:
import apache_beam as beam
print('Apache Beam version: {}'.format(beam.__version__))

import tensorflow as tf
print('Tensorflow version: {}'.format(tf.__version__))

import tensorflow_transform as tft
from tensorflow_transform import beam as tft_beam
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import schema_utils
print('TensorFlow Transform version: {}'.format(tft.__version__))


## Download the Data

A continuación, descargará los datos y los colocará en su espacio de trabajo.

In [ ]:
import os

# Directorio de los archivos de datos brutos
DATA_DIR = '/content/data/'

# Descargar el conjunto de datos
!wget -nc https://github.com/https-deeplearning-ai/machine-learning-engineering-for-production-public/raw/main/course2/week4-ungraded-lab/data/WISDM_ar_latest.tar.gz -P {DATA_DIR}

# Extraer el conjunto de datos
!tar -xvf {DATA_DIR}/WISDM_ar_latest.tar.gz -C {DATA_DIR}

# Asignar una ruta de datos a una variable para facilitar su consulta
INPUT_FILE = os.path.join(DATA_DIR, 'WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt')

## Inspect the Data

You should now inspect the dataset and you can start by previewing it as a dataframe.

In [ ]:
import pandas as pd

# Poner el conjunto de datos en un marco de datos
df = pd.read_csv(INPUT_FILE, header=None, names=['user_id', 'activity', 'timestamp', 'x-acc','y-acc', 'z-acc'])

# Vista previa de las primeras filas
df.head()

Es posible que note el punto y coma al final de los elementos `z-acc`. Esto puede hacer que los elementos se traten como una cadena, por lo que puede querer eliminarlo cuando analice sus datos. Esto lo hará más adelante en el proceso con [Beam.map()](https://beam.apache.org/documentation/transforms/python/elementwise/map/). De esto también se encarga la función `visualize_plots()` que se utilizará en la siguiente sección.

In [ ]:
# Visulaization Utilities
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def visualize_value_plots_for_categorical_feature(feature, colors=['b']):
    '''Plots a bar graph for categorical features'''
    counts = feature.value_counts()
    plt.bar(counts.index, counts.values, color=colors)
    plt.show()


def visualize_plots(dataset, activity, columns):
    '''Visualizes the accelerometer data against time'''
    features = dataset[dataset['activity'] == activity][columns][:200]
    if 'z-acc' in columns:
        # remove semicolons in the z-acc column
        features['z-acc'] = features['z-acc'].replace(regex=True, to_replace=r';', value=r'')
        features['z-acc'] = features['z-acc'].astype(np.float64)
    axis = features.plot(subplots=True, figsize=(16, 12), 
                     title=activity)

    for ax in axis:
        ax.legend(loc='lower left', bbox_to_anchor=(1.0, 0.5))

### Histogram of Activities

Ahora puede proceder a las visualizaciones. Puede trazar el histograma de actividades y hacer sus observaciones. Por ejemplo, te darás cuenta de que hay más datos para caminar y trotar que para otras actividades. Esto puede tener un efecto en la forma en que su modelo aprende cada actividad, por lo que debe tomar nota de ello. Por ejemplo, puede que quieras recoger más datos para las otras actividades.

In [ ]:
# Trazar el histograma de actividades
visualize_value_plots_for_categorical_feature(df['activity'], colors=['r', 'g', 'b', 'y', 'm', 'c'])

### Histograma de mediciones por usuario
También puede observar el número de mediciones realizadas por usuario.

In [ ]:
# Plot the histogram for users
visualize_value_plots_for_categorical_feature(df['user_id'])

Puedes consultar con los expertos en la materia cuál de los usuarios debe formar parte del conjunto de entrenamiento o de prueba. Para este laboratorio, sólo harás una simple partición. Pondrás los identificadores de usuario del 1 al 30 en el conjunto de entrenamiento, y el resto en el conjunto de prueba. Aquí está el `partition_fn` que usarás para `Beam.Partition()` más tarde.

In [ ]:
def partition_fn(line, num_partitions):
  '''
  Función de partición para trabajar con Beam.partition

  Args:
    line (string) - Un registro en el archivo CSV.
    num_partition (integer) - Número de particiones. Argumento requerido por Beam. No se utiliza en esta función.

  Devuelve:
    0 o 1 (integer) - 0 si el ID de usuario es menor de 30, 1 en caso contrario. 
  '''
  
  # Obtiene la primera subcadena delimitada por una coma. Convertir en un int.
  user_id = int(line[:line.index(b',')])

  # Comprueba si está por encima o por debajo de 30
  partition_num = int(user_id <= 30)

  return partition_num

### Acceleration per Activity

Por último, puedes trazar las mediciones de los sensores frente a las marcas de tiempo. Puedes observar que la aceleración es mayor en actividades como el jogging en comparación con estar sentado, lo que debería ser el comportamiento esperado. Si este no es el caso, entonces puede haber problemas con el sensor y puede hacer que los datos no sean válidos.

In [ ]:
# Plot the measurements for `Jogging`
visualize_plots(df, 'Jogging', columns=['x-acc', 'y-acc', 'z-acc'])

In [ ]:
visualize_plots(df, 'Sitting', columns=['x-acc', 'y-acc', 'z-acc'])

## Declare Schema for Cleaned Data

Como siempre, querrá declarar un esquema para asegurarse de que su entrada de datos se analiza correctamente.

In [ ]:
STRING_FEATURES = ['activity']
INT_FEATURES = ['user_id', 'timestamp']
FLOAT_FEATURES = ['x-acc', 'y-acc', 'z-acc']

# Declare feature spec
RAW_DATA_FEATURE_SPEC = dict(
    [(name, tf.io.FixedLenFeature([], tf.string))
     for name in STRING_FEATURES] +
    [(name, tf.io.FixedLenFeature([], tf.int64))
     for name in INT_FEATURES] +
    [(name, tf.io.FixedLenFeature([], tf.float32))
     for name in FLOAT_FEATURES]
)

# Create schema from feature spec
RAW_DATA_SCHEMA = tft.tf_metadata.schema_utils.schema_from_feature_spec(RAW_DATA_FEATURE_SPEC)

## Create a `tf.Transform` preprocessing_fn

A continuación, puede definir su función de preprocesamiento. Para este ejercicio, escalarás las características flotantes por sus valores min-max y crearás una búsqueda de vocabulario para la etiqueta de la cadena. También descartará las características que no necesitará en el modelo, como el ID de usuario y la marca de tiempo.

In [ ]:
LABEL_KEY = 'activity'

def preprocessing_fn(inputs):
  """Preprocess input columns into transformed columns."""

  # Copiar entradas
  outputs = inputs.copy()

  # Eliminar las características que no deben incluirse como entradas en el modelo
  del outputs["user_id"]
  del outputs["timestamp"]
  
  # Crear un vocabulario para las etiquetas de las cadenas
  outputs[LABEL_KEY] = tft.compute_and_apply_vocabulary(inputs[LABEL_KEY],vocab_filename=LABEL_KEY)

  # Escala las características por su mínimo-máximo
  for key in FLOAT_FEATURES:
     outputs[key] = tft.scale_by_min_max(outputs[key])

  return outputs

## Transform the data

Ahora estás listo para empezar a transformar los datos en un pipeline de Apache Beam usando Tensorflow Transform. Seguirá los siguientes pasos principales:

1. Lee los datos usando `beam.io.ReadFromText`.
1. 2. Limpiarlos usando `beam.Map` y `beam.Filter`.
1. 2. Transformarlos usando un pipeline de preprocesamiento que escala los datos numéricos y convierte los datos categóricos de cadenas a índices de valores int64. 
1. Escribe el resultado como un `TFRecord` de protos `Example`, que puede ser utilizado para entrenar un modelo más tarde.

In [ ]:
import shutil
from tfx_bsl.coders.example_coder import RecordBatchToExamplesEncoder
from tfx_bsl.public import tfxio

# Nombres de directorios para las salidas de TF Transform
WORKING_DIR = 'transform_dir'
TRANSFORM_TRAIN_FILENAME = 'transform_train'
TRANSFORM_TEST_FILENAME = 'transform_test'
TRANSFORM_TEMP_DIR = 'tft_temp'

ordered_columns = ['user_id', 'activity', 'timestamp', 'x-acc','y-acc', 'z-acc']

def transform_data(working_dir):
    '''
    Reads a CSV File and preprocesses the data using TF Transform

    Args:
      working_dir (string) - directory to place TF Transform outputs
    
    Returns:
      transform_fn - transformation graph
      transformed_train_data - transformed training examples
      transformed_test_data - transformed test examples
      transformed_metadata - transform output metadata
    '''

    # Eliminar TF Transform si ya existe
    if os.path.exists(working_dir):
      shutil.rmtree(working_dir)

    with beam.Pipeline() as pipeline:
        with tft_beam.Context(temp_dir=os.path.join(working_dir, TRANSFORM_TEMP_DIR)):
  
          # Leer el CSV de entrada, limpiar y filtrar los datos (sustituir el punto y coma y las filas incompletas)
          raw_data = (
              pipeline
              | 'ReadTrainData' >> beam.io.ReadFromText(INPUT_FILE, coder=beam.coders.BytesCoder())
              | 'CleanLines' >> beam.Map(lambda line: line.replace(b',;', b'').replace(b';', b''))
              | 'FilterLines' >> beam.Filter(lambda line: line.count(b',') == len(ordered_columns) - 1 and line[-1:] != b','))

          # Partición de los datos en datos de entrenamiento y de prueba utilizando beam.Partition
          raw_train_data, raw_test_data = (raw_data
                                  | 'TrainTestSplit' >> beam.Partition(partition_fn, 2))
                    
          # Crear un TFXIO para leer los datos con el esquema. 
          csv_tfxio = tfxio.BeamRecordCsvTFXIO(
              physical_format='text',
              column_names=ordered_columns,
              schema=RAW_DATA_SCHEMA)

          # Analizar los datos brutos del tren en entradas para la Transformación TF
          raw_train_data = (raw_train_data 
                            | 'DecodeTrainData' >> csv_tfxio.BeamSource())

          # Obtener los metadatos de los datos brutos
          RAW_DATA_METADATA = csv_tfxio.TensorAdapterConfig()
          
          # Emparejar los datos de la prueba con los metadatos en una tupla
          raw_train_dataset = (raw_train_data, RAW_DATA_METADATA)

          # Transformación de los datos de entrenamiento. El formato de salida TFXIO (RecordBatch)
          # se elige para mejorar el rendimiento.
          (transformed_train_data,transformed_metadata) , transform_fn = (
              raw_train_dataset 
                | 'AnalyzeAndTransformTrainData' >> tft_beam.AnalyzeAndTransformDataset(preprocessing_fn, output_record_batches=True))
          
          # # Analizar los datos de prueba en bruto en entradas para TF Transform
          raw_test_data = (raw_test_data 
                            |'DecodeTestData' >> csv_tfxio.BeamSource())

          # Emparejar los datos de la prueba con los metadatos en una tupla
          raw_test_dataset = (raw_test_data, RAW_DATA_METADATA)

          # Ahora aplique la misma función de transformación a los datos de prueba.
          # No necesitas el esquema de datos transformados. Es el mismo que antes.pñllllllllllllllllllllllllllllllllllllll
          transformed_test_data, _ = ((raw_test_dataset, transform_fn) 
                                        | 'AnalyzeAndTransformTestData' >> tft_beam.TransformDataset(output_record_batches=True))
          
          # Declare an encoder to convert output record batches to TF Examples 
          transformed_data_coder = RecordBatchToExamplesEncoder(transformed_metadata.schema)

          
          # Encode transformed train data and write to disk
          _ = (
              transformed_train_data
              | 'EncodeTrainData' >> beam.FlatMapTuple(lambda batch, _: transformed_data_coder.encode(batch))
              | 'WriteTrainData' >> beam.io.WriteToTFRecord(
                  os.path.join(working_dir, TRANSFORM_TRAIN_FILENAME)))

          # Encode transformed test data and write to disk
          _ = (
              transformed_test_data
              | 'EncodeTestData' >> beam.FlatMapTuple(lambda batch, _: transformed_data_coder.encode(batch))
              | 'WriteTestData' >> beam.io.WriteToTFRecord(
                  os.path.join(working_dir, TRANSFORM_TEST_FILENAME)))
          
          # Write transform function to disk
          _ = (
            transform_fn
            | 'WriteTransformFn' >>
            tft_beam.WriteTransformFn(os.path.join(working_dir)))

    return transform_fn, transformed_train_data, transformed_test_data, transformed_metadata

In [ ]:
def main():
  return transform_data(WORKING_DIR)

if __name__ == '__main__':
  transform_fn, transformed_train_data,transformed_test_data, transformed_metadata = main()


##Prepare Training and Test Datasets from TFTransformOutput

Ahora que tiene los ejemplos transformados, necesita preparar las ventanas del conjunto de datos para estos datos de series temporales. Como se discutió en clase, usted quiere agrupar una serie de mediciones y eso será la característica para una etiqueta particular. En este caso particular, tiene sentido agrupar mediciones consecutivas y usar eso como el indicador de una actividad. Por ejemplo, si se toman 80 mediciones y se oscila mucho (al igual que en las visualizaciones en las partes anteriores de este cuaderno), entonces el modelo debe ser capaz de decir que se trata de una actividad "Corriendo". Vamos a implementar esto en las siguientes celdas utilizando el método [tf.data.Dataset.window()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#window). Observa que hay una función `add_mode()`. Si recuerdas el aspecto del CSV original, verás que cada fila tiene una etiqueta de actividad. Por lo tanto, si queremos asociar una sola actividad a un grupo de 80 mediciones, entonces simplemente obtenemos la actividad que más se menciona en todas esas filas (por ejemplo, si 75 elementos de la ventana apuntan a la actividad `Caminando` y sólo 5 apuntan a `Sentado`, entonces toda la ventana se asocia a `Caminando`).

In [ ]:
# Obtener la salida del componente Transformación
tf_transform_output = tft.TFTransformOutput(os.path.join(WORKING_DIR))

# Parameters
HISTORY_SIZE = 80
BATCH_SIZE = 100
STEP_SIZE = 40

def parse_function(example_proto):
    '''Parse the values from tf examples'''
    feature_spec = tf_transform_output.transformed_feature_spec()
    features = tf.io.parse_single_example(example_proto, feature_spec)
    values = list(features.values())
    values = [float(value) for value in values]
    features = tf.stack(values, axis=0)
    return features

def add_mode(features):
    '''Calculate mode of activity for the current history size of elements'''
    unique, _, count = tf.unique_with_counts(features[:,0])
    max_occurrences = tf.reduce_max(count)
    max_cond = tf.equal(count, max_occurrences)
    max_numbers = tf.squeeze(tf.gather(unique, tf.where(max_cond)))

    #Características (X) son todas las características excepto la actividad (x-acc, y-acc, z-acc)
    #Objetivo(Y) es el modo de los valores de actividad de todas las filas de esta ventana
    return (features[:,1:], max_numbers)

def get_windowed_dataset(path):
  '''Get the dataset and group them into windows'''
  dataset = tf.data.TFRecordDataset(path)
  dataset = dataset.map(parse_function)
  dataset = dataset.window(HISTORY_SIZE, shift=STEP_SIZE, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(HISTORY_SIZE))
  dataset = dataset.map(add_mode)
  dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
  dataset = dataset.repeat()

  return dataset

In [ ]:
# Obtiene la lista de nombres de archivos de datos de entrenamiento y de prueba tfrecord de las salidas de la transformación
train_tfrecord_files = tf.io.gfile.glob(os.path.join(WORKING_DIR, TRANSFORM_TRAIN_FILENAME + '*'))
test_tfrecord_files = tf.io.gfile.glob(os.path.join(WORKING_DIR, TRANSFORM_TEST_FILENAME + '*'))

# Generate dataset windows
windowed_train_dataset = get_windowed_dataset(train_tfrecord_files[0])
windowed_test_dataset = get_windowed_dataset(test_tfrecord_files[0])

In [ ]:
# Vista previa de un ejemplo en el conjunto de datos de entrenamiento
for x, y in windowed_train_dataset.take(1):
  print("\nFeatures (x-acc, y-acc, z-acc):\n")
  print(x)
  print("\nTarget (activity):\n")
  print(y)

Debería ver una muestra de la ventana de un conjunto de datos en la parte superior. Hay 80 mediciones consecutivas de `x-acc`, `y-acc` y `z-acc` que corresponden a una sola actividad etiquetada. Además, también se ha configurado para que haya lotes de 100 ventanas. Esto puede ser alimentado para entrenar un LSTM para que pueda aprender a detectar actividades basadas en 80 ventanas de medición. También se puede previsualizar una muestra en el conjunto de pruebas:

In [ ]:
# Preview an example in the train dataset
for x, y in windowed_test_dataset.take(1):
  print("\nFeatures (x-acc, y-acc, z-acc):\n")
  print(x)
  print("\nTarget (activity):\n")
  print(y)

## Wrap Up

En este laboratorio, ha podido preparar datos de series temporales de un acelerómetro para transformar características que se agrupan en ventanas para hacer predicciones. El mismo concepto puede aplicarse a cualquier dato en el que necesites tomar unos segundos de mediciones antes de que el modelo haga una predicción. 